# The two-sites Hubbard model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh_tridiagonal

In [2]:
from qutip import tensor, destroy, qeye

N = 10

aL = tensor(destroy(N), qeye(N))
aR = tensor(qeye(N), destroy(N))
num_L = aL.dag() * aL
num_R = aR.dag() * aR

U = 1.

H = -(aL.dag() * aR + aR.dag() * aL) + U * (num_L * (num_L - 1) + num_R * (num_R - 1))

In [3]:
H.eigenenergies()[:20]

array([-1.23606798, -1.        , -0.46410162,  0.        ,  1.        ,
        1.25543735,  2.        ,  3.        ,  3.23606798,  3.96425886,
        5.39444872,  6.46410162,  7.        ,  7.6521596 ,  8.        ,
        8.44109537, 12.32994651, 12.41881243, 12.58555378, 12.60555128])

In [4]:
def Cantor(x, y):
    return (x + y) * (x + y + 1) / 2 + y


z_max = Cantor(N, N)
z = np.arange(0, z_max)

w = np.floor(0.5 * (np.sqrt(8 * z + 1) - 1))
y = z - w * (w + 1) / 2
x = w - y

nL = x.astype(int)
nR = y.astype(int)

In [5]:
np.max(
    np.abs(
        eigh_tridiagonal(
            U * (nL * (nL - 1) + nR * (nR - 1)), -np.sqrt((nL + 1) * nR)[1::],
            eigvals_only = True
        )[:15] - H.eigenenergies()[:15]
    )
)

3.241851231905457e-14